In [2]:
%load_ext autoreload
%autoreload 2
#%reload_ext autoreload

# temporary solution for relative imports in case sadl is not installed
# if sadl is installed, no need to use the line
import sys
import os
import inspect
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
from inspect import signature
import numpy as np
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))

from SADL.static_data.algorithms import pyod

kwargs = {"algorithm_": "cblof","contamination":2, "label_parser": True}
model = pyod.PyodAnomalyDetection(**kwargs)

#print(model.algorithm_)
print(model.label_parser)
print(model.get_params())
#print(signature(model.__class__).parameters.values())

kwargs = {"algorithm_": "abod","contamination":2, "label_parser": False}
model.set_params(**kwargs)
print(model.label_parser)
print(model.get_params())


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
True
{'label_parser': True, 'algorithm_': 'CBLOF', 'alpha': 0.9, 'beta': 5, 'check_estimator': False, 'clustering_estimator': None, 'contamination': 2, 'n_clusters': 8, 'random_state': None, 'use_weights': False}
False
{'label_parser': False, 'algorithm_': 'ABOD', 'contamination': 2, 'method': 'fast', 'n_neighbors': 5}


Data generation

In [3]:
# Define the number of inliers and outliers
n_samples = 200
outliers_fraction = 0.25

# Compare given detectors under given settings
# Initialize the data
xx, yy = np.meshgrid(np.linspace(-7, 7, 100), np.linspace(-7, 7, 100))
n_inliers = int((1. - outliers_fraction) * n_samples)
n_outliers = int(outliers_fraction * n_samples)
ground_truth = np.zeros(n_samples, dtype=int)
ground_truth[-n_outliers:] = 1

# Show the statics of the data
print('Number of inliers: %i' % n_inliers)
print('Number of outliers: %i' % n_outliers)
print('Ground truth shape is {shape}. Outlier are 1 and inliers are 0.\n'.format(shape=ground_truth.shape))
print(ground_truth)

X1 = 0.3 * np.random.randn(n_inliers // 2, 2)
X2 = 0.3 * np.random.randn(n_inliers // 2, 2)
X = np.r_[X1, X2]
# Add outliers
X = np.r_[X, np.random.uniform(low=-6, high=6, size=(n_outliers, 2))]


Number of inliers: 150
Number of outliers: 50
Ground truth shape is (200,). Outlier are 1 and inliers are 0.

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


Model Fitting - errors


In [4]:
%load_ext autoreload
%autoreload 2
kwargs = {"algorithm_": "abod","contamination":2, "label_parser": True, "n_neighbors": 4, "method": 'method'}
model = pyod.PyodAnomalyDetection(**kwargs)

# Fit the model using SADL Library - error en fit
model.fit(X)
kwargs = {"algorithm_": "abod","contamination":0.1, "label_parser": True, "n_neighbors": 4, "method": 'fast'}
model.set_params(**kwargs)
#print(model.model)
# Fit the model using SADL Library - sin error
#model.fit(X)
scores_pred = model.decision_function(X) * -1
print(scores_pred)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
PYODerror fit(): ('method', 'is not a valid method')
For further reference please see: https://pyod.readthedocs.io/en/latest/
PYODerror decision_function(): This ABOD instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.
For further reference please see: https://pyod.readthedocs.io/en/latest/


TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'

Model Fitting - OK


In [5]:
%load_ext autoreload
%autoreload 2
kwargs = {"algorithm_": "abod","contamination":0.1, "label_parser": True, "n_neighbors": 4, "method": 'fast'}
model.set_params(**kwargs)

print(model.model)
# Fit the model using SADL Library - sin error
model.fit(X)
scores_pred = model.decision_function(X) * -1
print(scores_pred)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
ABOD(contamination=0.1, method='fast', n_neighbors=4)
[1.53538665e+04 1.23094698e+04 4.03161056e+03 1.32013400e+05
 8.51512296e+05 6.90482407e+03 2.35396553e+04 1.32268523e+05
 1.15574602e+03 2.38474354e+02 2.30630956e+04 4.94908026e+03
 1.27617902e+03 1.87414113e+05 1.40413844e+06 1.74327665e+00
 3.47678726e+03 1.31154212e+07 5.05920000e+05 2.65628836e+03
 1.45008084e+03 1.91919246e+03 1.58539498e+05 3.55191643e+05
 1.22432454e+04 8.86529095e+05 3.77813092e+05 1.94115933e+02
 5.60551514e+04 3.99661008e+05 1.07819559e+04 1.48851747e+03
 1.11333226e+05 1.39272403e+03 4.68483485e+04 5.60969486e+03
 1.55615139e+05 1.04219153e+05 1.57114267e+04 6.96177623e+04
 3.43934105e+01 6.61455339e+04 9.65492134e+03 5.41557754e+03
 1.32942397e+05 9.92568295e+04 4.14066061e+03 1.96118193e+03
 9.43116457e+02 1.01509127e+06 5.27969617e+04 1.22932386e+07
 4.99646146e+03 9.40840505e+03 2.96972314e+04 4.92003022e+04
 2.7

Model Predict w/ label_parser function

In [6]:
%load_ext autoreload
%autoreload 2

def function(X):
    print("Estoy en funcion")
    return X < 0.5

kwargs = {"algorithm_": "abod","contamination":0.1, "label_parser": function, "n_neighbors": 4, "method": 'fast'}
model.set_params(**kwargs)

print(model.get_params())
model.predict(X)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'label_parser': <function function at 0x7f2d47aace50>, 'algorithm_': 'ABOD', 'contamination': 0.1, 'method': 'fast', 'n_neighbors': 4}
Estoy en funcion


array([[ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True, False],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [False,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True, False],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True, False],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ Tr

Model Predict wo/ label_parser

In [7]:
%load_ext autoreload
%autoreload 2


def function(X):
    print("Estoy en funcion")
    return X < 0.5

kwargs = {"algorithm_": "abod","contamination":0.1, "label_parser": None, "n_neighbors": 4, "method": 'fast'}
model.set_params(**kwargs)

print(model.get_params().keys())
model.fit(X)
model.predict(X)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
dict_keys(['label_parser', 'algorithm_', 'contamination', 'method', 'n_neighbors'])


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1])

In [8]:
%load_ext autoreload
%autoreload 2

import sys
import os
from pyod.models.lscp import LSCP
from pyod.models.lof import LOF
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
from SADL.static_data.algorithms.pyod import PyodAnomalyDetection

kwargs_lof = {"algorithm_": "lof"}

detector_list = [pyod.PyodAnomalyDetection(**kwargs_lof).model, pyod.PyodAnomalyDetection(**kwargs_lof).model]

kwargs = {"algorithm_": "lscp", "detector_list":detector_list,"random_state": 42}
model = pyod.PyodAnomalyDetection(**kwargs)
print(model.get_params())


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'label_parser': None, 'algorithm_': 'LSCP', 'contamination': 0.1, 'detector_list': [LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=20, novelty=True, p=2), LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=20, novelty=True, p=2)], 'local_max_features': 1.0, 'local_region_size': 30, 'n_bins': 10, 'random_state': 42}
